# BETA Dataset Inspector 


This notebook is a tool for exploring the structure and contents of the BETA dataset.

**How to use**
1. Make sure this notebook is in the root directory of your project.
2. Set the `SUBJECT_ID_TO_INSPECT` variable in the cell below.
3. Run all the cells to see a full breakdown of the data structure for that subject.

In [2]:
import scipy.io
import numpy as np
import os

### 1. Configuration
Set the path to your data and the subject ID you want to inspect.

In [3]:
# This should be the path to the folder containing S1.mat, S2.mat, etc.
# Based on your directory structure, it should be 'Data'.
DATA_PATH = '../Data'

# Change this number to inspect any subject from 1 to 70.
SUBJECT_ID = 1

### 2. Helper Function for Data Inspection
This function recursively prints the structure of the loaded MATLAB data, similar to a `pandas.info()` summary but for nested structures.

In [6]:
def inspect_mat_structure(data, indent=0):
    """Recursively prints the keys, types, and shapes of a loaded .mat file."""
    
    # Determine if the data is a dictionary (from top-level) or a mat_struct object
    if isinstance(data, dict):
        items = data.items()
    elif isinstance(data, scipy.io.matlab.mat_struct):
        # Get the field names from the object's dictionary, ignoring private/magic methods
        items = {key: getattr(data, key) for key in data.__dict__ if not key.startswith('_')}.items()
    else:
        return
    
    for key, value in items:
        # Indent for readability
        print('  ' * indent, end='')
        
        if isinstance(value, scipy.io.matlab.mat_struct):
            print(f"Key/Field: '{key}' | Type: MATLAB Struct Object")
            # Recursively call the function on the struct object itself
            inspect_mat_structure(value, indent + 1)
            
        elif isinstance(value, np.ndarray):
            print(f"Key/Field: '{key}' | Type: numpy.ndarray | Shape: {value.shape} | Dtype: {value.dtype}")
        
        else:
            # For simple variables like strings, numbers, etc.
            print(f"Key/Field: '{key}' | Type: {type(value).__name__} | Value: {value}")

### 3. Load and Inspect the Data
This cell loads the specified subject's data and runs the inspection function.
The output will show you the exact keys and structure you need to use in your `test.py` script.


In [7]:
if __name__ == "__main__":
    file_path = os.path.join(DATA_PATH, f'S{SUBJECT_ID}.mat')
    mat_data = None

    try:
        print(f"--- Inspecting file: {file_path} ---\n")
        # struct_as_record=False is important to load structs as objects for easier inspection
        # squeeze_me=True removes singleton dimensions, which is convenient
        mat_data = scipy.io.loadmat(file_path, squeeze_me=True, struct_as_record=False)
        
        print("--- Full file structure ---")
        # Run the inspection
        inspect_mat_structure(mat_data)

    except FileNotFoundError:
        print(f"ERROR: File not found at '{file_path}'. Please check your DATA_PATH variable.")
    except Exception as e:
        print(f"An error occurred: {e}")

--- Inspecting file: ../Data/S1.mat ---

--- Full file structure ---
Key/Field: '__header__' | Type: bytes | Value: b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Wed Oct 20 11:40:24 2021'
Key/Field: '__version__' | Type: str | Value: 1.0
Key/Field: '__globals__' | Type: list | Value: []
Key/Field: 'data' | Type: MATLAB Struct Object
  Key/Field: 'EEG' | Type: numpy.ndarray | Shape: (64, 750, 4, 40) | Dtype: float64
  Key/Field: 'suppl_info' | Type: MATLAB Struct Object
    Key/Field: 'sub' | Type: str | Value: S1
    Key/Field: 'age' | Type: float | Value: 26.5
    Key/Field: 'gender' | Type: str | Value: female
    Key/Field: 'chan' | Type: numpy.ndarray | Shape: (64, 4) | Dtype: object
    Key/Field: 'freqs' | Type: numpy.ndarray | Shape: (40,) | Dtype: float64
    Key/Field: 'phases' | Type: numpy.ndarray | Shape: (40,) | Dtype: float64
    Key/Field: 'bci_quotient' | Type: float | Value: 108.44332637198063
    Key/Field: 'wide_snr' | Type: float | Value: -12.479727738714981

### 4. Specific Inspection of the 'data' Field
This section specifically isolates the 'data' struct and inspects its contents.
This will show you exactly what attributes it has.

In [8]:
if mat_data and 'data' in mat_data:
    print("\n--- Inspecting Fields within the 'data' Struct ---")
    
    try:
        # Access the mat_struct object
        data_struct = mat_data['data']
        
        # Use the same inspection function to see what's inside
        inspect_mat_structure(data_struct)
    except Exception as e:
        print(f"Could not inspect the 'data' struct. Error: {e}")
else:
    print("\n--- The key 'data' was not found at the top level of the .mat file. ---")


--- Inspecting Fields within the 'data' Struct ---
Key/Field: 'EEG' | Type: numpy.ndarray | Shape: (64, 750, 4, 40) | Dtype: float64
Key/Field: 'suppl_info' | Type: MATLAB Struct Object
  Key/Field: 'sub' | Type: str | Value: S1
  Key/Field: 'age' | Type: float | Value: 26.5
  Key/Field: 'gender' | Type: str | Value: female
  Key/Field: 'chan' | Type: numpy.ndarray | Shape: (64, 4) | Dtype: object
  Key/Field: 'freqs' | Type: numpy.ndarray | Shape: (40,) | Dtype: float64
  Key/Field: 'phases' | Type: numpy.ndarray | Shape: (40,) | Dtype: float64
  Key/Field: 'bci_quotient' | Type: float | Value: 108.44332637198063
  Key/Field: 'wide_snr' | Type: float | Value: -12.479727738714981
  Key/Field: 'narrow_snr' | Type: float | Value: 4.671157504218424
  Key/Field: 'srate' | Type: int | Value: 250
